<a href="https://colab.research.google.com/github/gokhanturer/NER_Model_SparkNLP/blob/main/Copy_of_BioNLP09_IOB_to_CoNLL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
pd.set_option("max_rows",100)

In [ ]:
!wget -q https://raw.githubusercontent.com/cambridgeltl/MTL-Bioinformatics-2016/master/data/BioNLP09-IOB/train.tsv
!wget -q https://raw.githubusercontent.com/cambridgeltl/MTL-Bioinformatics-2016/master/data/BioNLP09-IOB/test.tsv

In [ ]:
with open("train.tsv") as f:
  train_data = f.read()

In [ ]:
with open("test.tsv") as f:
  test_data = f.read()

In [ ]:
def data_pre_process(data):

  result = []

  for i in data.split("\n"):
    if i == "": # Each sentence ends ""
      result.append("None\tNone") # We add "None" between each sentences
    else:
      result.append(i)
  
  df = pd.DataFrame(result)

  return df

In [ ]:
def conll_generator(data, filename):

  data = data_pre_process(data)

  data.to_csv(filename) # We saved the return value of the data_prepare_function as csv file

  df = pd.read_csv(filename, sep = "\t", usecols = [0,1],names = ['token','entity'])

  # We read csv file , add column name and assigned a NaN value to first row 
  df.iloc[0,:] = np.nan
  
  # After reading the data, we cleaned the unnecessary characters in the columns.
  df["token"] = df["token"].map(lambda x: x.split(",")[1], na_action = "ignore")
  df["token"] = df["token"].map(lambda x: ',' if x == '"' else x, na_action = "ignore")
  df["entity"] = df["entity"].map(lambda x: x.strip('"'), na_action = "ignore")

  # We assigned the value np.nan to the places that are None
  df["token"]= df["token"].map(lambda x: np.nan if x == 'None' else x)
  df["entity"] = df["entity"].map(lambda x: np.nan if x == 'None' else x)

  # We added two new columns with value "NN" excluding NaN values
  df["pos1"] = df["token"].map(lambda x: np.nan if type(x)== float else 'NN')
  df["pos2"] = df["token"].map(lambda x: np.nan if type(x)== float else 'NN')

  df = df[["token","pos1","pos2","entity"]]

  # We changed the column names according to the conll format
  df.columns = ["-DOCSTART-", "-X-", "-X-", "O"]

  return df

In [ ]:
data = train_data

filename = "train_data.csv"

train_data = conll_generator(data, filename)

In [ ]:
train_data.to_csv("/content/BioNLP09_IOB_train.conll", index = False, sep = " ")

In [ ]:
data = test_data

filename = "test_data.csv"

test_data = conll_generator(data, filename)

In [ ]:
test_data.to_csv("/content/BioNLP09_IOB_test.conll", index = False, sep = " ")